# Analyse TagesWoche mit Google Analytics Daten und API Metadaten



In [1]:
%matplotlib inline

from IPython.core.interactiveshell import InteractiveShell
from datetime import datetime

import os

import glob
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib
import matplotlib.pyplot as plt
from matplotlib.backends.backend_pdf import PdfPages
from pandas.io.json import json_normalize

InteractiveShell.ast_node_interactivity = "all"
matplotlib.rcParams['svg.fonttype'] = 'none'

In [88]:
## define month

month = 'juli_2016'

In [89]:
## load google analyctics data

ga_stats = 'tawo_stats_' + month + '.csv'

stats = pd.read_csv(ga_stats)

stats['article_id'] = stats['Seite'].str.replace('/\w\w/\d\d\d\d_\d\d/\w+/', '')
stats['Seite'] = stats['Seite'].str.replace('/\w\w/\d\d\d\d_\d\d/', '')
stats['section'] = stats['Seite'].str.extract('(\w+)')

stats['article_id'] = pd.to_numeric(stats['article_id'], errors='coerce')
stats['Seitenaufrufe'] = pd.to_numeric(stats['Seitenaufrufe'], errors='coerce')

stats.replace('', np.nan, inplace=True)
stats = stats.dropna()

stats.head()




/Library/Frameworks/Python.framework/Versions/3.5/lib/python3.5/site-packages/ipykernel/__main__.py:9: FutureWarning: currently extract(expand=None) means expand=False (return Index/Series/DataFrame) but in a future version of pandas this will be changed to expand=True (return DataFrame)


,Seite,Seitenaufrufe,Einzelne Seitenaufrufe,Durchschn Besuchszeit auf Seite,Einstiege,Absprungrate,% Ausstiege,Seitenwert,article_id,section
0,international/724751,30472.0,28207.0,00:05:20,26107.0,"89,41 %","84,49 %","0,00 CHF",724751.0,international
1,basel/725295,18516.0,17199.0,00:04:07,14976.0,"89,02 %","81,48 %","0,00 CHF",725295.0,basel
2,basel/724533,13785.0,12983.0,00:05:56,10961.0,"90,72 %","82,55 %","0,00 CHF",724533.0,basel
3,basel/725275,11986.0,11184.0,00:05:10,10453.0,"91,11 %","86,72 %","0,00 CHF",725275.0,basel
4,international/724826,10741.0,10061.0,00:03:59,8923.0,"90,54 %","83,59 %","0,00 CHF",724826.0,international


In [90]:
def parse_articles(id):
    meta = pd.DataFrame(columns=['type', 'article_id'])
    url = 'http://www.tageswoche.ch/content-api/articles/'
    article_name = url + str(id)
    print(id)
    try:
        df = pd.read_json(article_name, lines=True)
        meta['type'] = df['type']
        meta['published'] = df['published']
        meta['article_id'] = id
    except:
        pass
        
    return meta

filepath = 'tawo_stats_meta_' + month + '.csv'

if os.path.exists(filepath):
    df = pd.read_csv(filepath, usecols=[1,2,3])
else:
    df = pd.concat([parse_articles(id) for id in stats['article_id']])


df

724751.0
725295.0
724533.0
725275.0
724826.0
725703.0
723646.0
723564.0
725129.0
724790.0
724127.0
724625.0
724047.0
724825.0
725305.0
723504.0
725227.0
725831.0
724381.0
725725.0
723559.0
724763.0
724693.0
724978.0
724718.0
724732.0
725019.0
724624.0
725198.0
725815.0
723452.0
725385.0
724611.0
725308.0
723566.0
724891.0
723861.0
724401.0
723567.0
725224.0
724816.0
724375.0
725786.0
723826.0
725023.0
725244.0
725647.0
724262.0
664044.0
725798.0
724459.0
725141.0
723840.0
724235.0
723992.0
725406.0
724530.0
723462.0
723854.0
724148.0
724034.0
725271.0
725045.0
725529.0
725386.0
725087.0
725897.0
724438.0
725475.0
717663.0
723533.0
724006.0
725478.0
725458.0
724853.0
723404.0
725185.0
677635.0
725826.0
665288.0
724666.0
725310.0
724682.0
723795.0
725508.0
666835.0
724672.0
725196.0
725058.0
724057.0
724638.0
723622.0
725046.0
672669.0
723750.0
724792.0
725668.0
705926.0
723494.0
725477.0
724157.0
724783.0
723912.0
665174.0
725330.0
725530.0
724974.0
725036.0
722946.0
724243.0
724599.0
7

,article_id,published,type
0,724751.0,2016-07-15T10:43:27+0200,news
0,725295.0,2016-07-22T16:55:04+0200,news
0,724533.0,2016-07-13T16:41:36+0200,news
0,725275.0,2016-07-26T04:50:05+0200,news
0,724826.0,2016-07-16T12:07:22+0200,news
0,725703.0,2016-07-29T04:50:05+0200,news
0,723646.0,2016-07-01T18:17:59+0200,news
0,723564.0,2016-07-01T04:50:06+0200,news
0,725129.0,2016-07-20T16:47:15+0200,news
0,724790.0,2016-07-15T14:55:03+0200,newswire


In [91]:
df['published'] = df['published'].str.extract('(\d\d\d\d-\d\d-\d\d)')

df = df[df.published != '0001-11-30']

df['published'] = pd.to_datetime(df['published'], format='%Y-%m-%d')

df

/Library/Frameworks/Python.framework/Versions/3.5/lib/python3.5/site-packages/ipykernel/__main__.py:1: FutureWarning: currently extract(expand=None) means expand=False (return Index/Series/DataFrame) but in a future version of pandas this will be changed to expand=True (return DataFrame)
  if __name__ == '__main__':
/Library/Frameworks/Python.framework/Versions/3.5/lib/python3.5/site-packages/ipykernel/__main__.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


,article_id,published,type
0,724751.0,2016-07-15,news
0,725295.0,2016-07-22,news
0,724533.0,2016-07-13,news
0,725275.0,2016-07-26,news
0,724826.0,2016-07-16,news
0,725703.0,2016-07-29,news
0,723646.0,2016-07-01,news
0,723564.0,2016-07-01,news
0,725129.0,2016-07-20,news
0,724790.0,2016-07-15,newswire


In [92]:
df.to_csv(filepath)

In [93]:
df = df.merge(stats, left_on='article_id', right_on='article_id')

df

,article_id,published,type,Seite,Seitenaufrufe,Einzelne Seitenaufrufe,Durchschn Besuchszeit auf Seite,Einstiege,Absprungrate,% Ausstiege,Seitenwert,section
0,724751.0,2016-07-15,news,international/724751,30472.0,28207.0,00:05:20,26107.0,"89,41 %","84,49 %","0,00 CHF",international
1,725295.0,2016-07-22,news,basel/725295,18516.0,17199.0,00:04:07,14976.0,"89,02 %","81,48 %","0,00 CHF",basel
2,724533.0,2016-07-13,news,basel/724533,13785.0,12983.0,00:05:56,10961.0,"90,72 %","82,55 %","0,00 CHF",basel
3,725275.0,2016-07-26,news,basel/725275,11986.0,11184.0,00:05:10,10453.0,"91,11 %","86,72 %","0,00 CHF",basel
4,724826.0,2016-07-16,news,international/724826,10741.0,10061.0,00:03:59,8923.0,"90,54 %","83,59 %","0,00 CHF",international
5,725703.0,2016-07-29,news,basel/725703,10112.0,9471.0,00:06:51,7948.0,"91,04 %","83,30 %","0,00 CHF",basel
6,723646.0,2016-07-01,news,kultur/723646,8810.0,8335.0,00:03:04,6546.0,"91,19 %","78,54 %","0,00 CHF",kultur
7,723564.0,2016-07-01,news,basel/723564,7870.0,7498.0,00:02:37,6278.0,"89,73 %","80,38 %","0,00 CHF",basel
8,725129.0,2016-07-20,news,basel/725129,7662.0,7183.0,00:02:55,6144.0,"90,77 %","81,04 %","0,00 CHF",basel
9,724790.0,2016-07-15,newswire,schweiz/724790,7365.0,6844.0,00:02:20,5564.0,"86,92 %","75,93 %","0,00 CHF",schweiz


In [94]:
scrape_file = 'ga_stats_merged_' + month + '.csv'

df.to_csv(scrape_file)

In [95]:
df['type'] = df['type'].astype('category')

df_news = df[(df.type == 'news')]

df_newswire = df[(df.type == 'newswire')]

df['type'].value_counts()

news             2456
newswire         1314
deb_moderator       6
blog                6
Name: type, dtype: int64

In [96]:
df_newswire['Seitenaufrufe'].sum()

78264.0

In [97]:
df_news['Seitenaufrufe'].sum()

568156.0

In [99]:
data_subset_news = df_news.set_index(['published'])

data_subset_news_month = data_subset_news.loc['2016-07-01':'2016-07-31']
data_subset_news_month['Seitenaufrufe'].sum()

453090.0

In [100]:
data_subset_news_month.sort_values('Seitenaufrufe', ascending=False).head()

,article_id,type,Seite,Seitenaufrufe,Einzelne Seitenaufrufe,Durchschn Besuchszeit auf Seite,Einstiege,Absprungrate,% Ausstiege,Seitenwert,section
published,,,,,,,,,,,
2016-07-15,724751.0,news,international/724751,30472.0,28207.0,00:05:20,26107.0,"89,41 %","84,49 %","0,00 CHF",international
2016-07-22,725295.0,news,basel/725295,18516.0,17199.0,00:04:07,14976.0,"89,02 %","81,48 %","0,00 CHF",basel
2016-07-13,724533.0,news,basel/724533,13785.0,12983.0,00:05:56,10961.0,"90,72 %","82,55 %","0,00 CHF",basel
2016-07-26,725275.0,news,basel/725275,11986.0,11184.0,00:05:10,10453.0,"91,11 %","86,72 %","0,00 CHF",basel
2016-07-16,724826.0,news,international/724826,10741.0,10061.0,00:03:59,8923.0,"90,54 %","83,59 %","0,00 CHF",international


In [101]:
data_subset_news_month.reset_index().sort_values('published')

data_subset_news_month.groupby(['section']).Seitenaufrufe.sum()

data_subset_news_month['section'].value_counts()

,published,article_id,type,Seite,Seitenaufrufe,Einzelne Seitenaufrufe,Durchschn Besuchszeit auf Seite,Einstiege,Absprungrate,% Ausstiege,Seitenwert,section
226,2016-07-01,723594.0,news,basel/723594,283.0,263.0,00:02:26,70.0,"65,71 %","49,12 %","0,00 CHF",basel
64,2016-07-01,723533.0,news,sport/723533,2085.0,1982.0,00:06:00,606.0,"79,54 %","56,55 %","0,00 CHF",sport
191,2016-07-01,723596.0,news,kultur/723596,482.0,449.0,00:04:04,180.0,"80,00 %","56,02 %","0,00 CHF",kultur
182,2016-07-01,723543.0,news,international/723543,547.0,506.0,00:01:51,228.0,"64,91 %","48,99 %","0,00 CHF",international
207,2016-07-01,723625.0,news,basel/723625,374.0,341.0,00:02:28,38.0,"68,42 %","39,57 %","0,00 CHF",basel
210,2016-07-01,723613.0,news,sport/723613,366.0,297.0,00:02:27,134.0,"88,81 %","56,83 %","0,00 CHF",sport
176,2016-07-01,723506.0,news,basel/723506,574.0,500.0,00:02:33,229.0,"64,63 %","43,73 %","0,00 CHF",basel
81,2016-07-01,723622.0,news,sport/723622,1676.0,1573.0,00:02:36,938.0,"86,57 %","68,97 %","0,00 CHF",sport
164,2016-07-01,723448.0,news,basel/723448,668.0,608.0,00:03:13,210.0,"71,43 %","42,96 %","0,00 CHF",basel
63,2016-07-01,717663.0,news,kultur/717663,2086.0,1918.0,00:04:57,1443.0,"87,11 %","74,98 %","0,00 CHF",kultur


section
basel            221075.0
international     65307.0
kultur            38090.0
leben             18382.0
schweiz            7452.0
sport            102784.0
Name: Seitenaufrufe, dtype: float64

basel            112
sport             59
kultur            40
international     26
leben             19
schweiz            6
Name: section, dtype: int64

In [103]:
data_subset_newswire = df_newswire.set_index(['published'])

data_subset_newswire_month = data_subset_newswire.loc['2016-07-01':'2016-07-31']

data_subset_newswire_month['Seitenaufrufe'].sum()

74021.0

In [104]:
data_subset_newswire_month.reset_index().sort_values('published').head()

data_subset_newswire_month.groupby(['section']).Seitenaufrufe.sum()

data_subset_newswire_month['section'].value_counts()


,published,article_id,type,Seite,Seitenaufrufe,Einzelne Seitenaufrufe,Durchschn Besuchszeit auf Seite,Einstiege,Absprungrate,% Ausstiege,Seitenwert,section
923,2016-07-01,723651.0,newswire,schweiz/723651,11.0,11.0,00:01:12,6.0,"100,00 %","63,64 %","0,00 CHF",schweiz
706,2016-07-01,723664.0,newswire,international/723664,15.0,15.0,00:01:52,0.0,"0,00 %","33,33 %","0,00 CHF",international
993,2016-07-01,723604.0,newswire,international/723604,10.0,10.0,00:01:02,0.0,"0,00 %","20,00 %","0,00 CHF",international
758,2016-07-01,723645.0,newswire,international/723645,14.0,14.0,00:02:54,2.0,"50,00 %","21,43 %","0,00 CHF",international
80,2016-07-01,723637.0,newswire,basel/723637,136.0,130.0,00:01:15,0.0,"0,00 %","19,85 %","0,00 CHF",basel


section
basel             7718.0
international    17764.0
kultur            3503.0
schweiz          26984.0
sport            18052.0
Name: Seitenaufrufe, dtype: float64

international    327
sport            326
schweiz          294
basel            143
kultur            71
Name: section, dtype: int64

In [105]:
data_subset_news_month = data_subset_news_month.reset_index()

In [106]:
data_subset_news_month.head()

,published,article_id,type,Seite,Seitenaufrufe,Einzelne Seitenaufrufe,Durchschn Besuchszeit auf Seite,Einstiege,Absprungrate,% Ausstiege,Seitenwert,section
0,2016-07-15,724751.0,news,international/724751,30472.0,28207.0,00:05:20,26107.0,"89,41 %","84,49 %","0,00 CHF",international
1,2016-07-22,725295.0,news,basel/725295,18516.0,17199.0,00:04:07,14976.0,"89,02 %","81,48 %","0,00 CHF",basel
2,2016-07-13,724533.0,news,basel/724533,13785.0,12983.0,00:05:56,10961.0,"90,72 %","82,55 %","0,00 CHF",basel
3,2016-07-26,725275.0,news,basel/725275,11986.0,11184.0,00:05:10,10453.0,"91,11 %","86,72 %","0,00 CHF",basel
4,2016-07-16,724826.0,news,international/724826,10741.0,10061.0,00:03:59,8923.0,"90,54 %","83,59 %","0,00 CHF",international


In [108]:
#data_subset_news_september.resample('D', how='count')


data_subset_news_month['week'] = data_subset_news_month['published'].dt.week

data_subset_news_month.head()


,published,article_id,type,Seite,Seitenaufrufe,Einzelne Seitenaufrufe,Durchschn Besuchszeit auf Seite,Einstiege,Absprungrate,% Ausstiege,Seitenwert,section,week
0,2016-07-15,724751.0,news,international/724751,30472.0,28207.0,00:05:20,26107.0,"89,41 %","84,49 %","0,00 CHF",international,28
1,2016-07-22,725295.0,news,basel/725295,18516.0,17199.0,00:04:07,14976.0,"89,02 %","81,48 %","0,00 CHF",basel,29
2,2016-07-13,724533.0,news,basel/724533,13785.0,12983.0,00:05:56,10961.0,"90,72 %","82,55 %","0,00 CHF",basel,28
3,2016-07-26,725275.0,news,basel/725275,11986.0,11184.0,00:05:10,10453.0,"91,11 %","86,72 %","0,00 CHF",basel,30
4,2016-07-16,724826.0,news,international/724826,10741.0,10061.0,00:03:59,8923.0,"90,54 %","83,59 %","0,00 CHF",international,28


In [111]:
data_subset_news_month.groupby(['week', 'section']).Seitenaufrufe.sum().to_csv("week_pageviews.csv")

### Scrape für Autoren

In [112]:
import urllib.request
import json
import codecs

def get_authors(id):
    result = pd.DataFrame(columns=['authors', 'id'])
    i = 0;
    url = 'http://www.tageswoche.ch/content-api/articles/' + str(id)
    try:
        response = urllib.request.urlopen(url)
        reader = codecs.getreader("utf-8")
        obj = json.load(reader(response))
        for i in range(0, len(obj['authors'])):
            result.loc[i] = obj['authors'][i]['name']
    except:
        result['authors'] = ''
        pass
    print(id)
    result['id'] = id
    return result #create a series


authors = pd.concat([get_authors(id) for id in data_subset_news_month['article_id']])

724751.0
725295.0
724533.0
725275.0
724826.0
725703.0
723646.0
723564.0
725129.0
724127.0
724625.0
724047.0
724825.0
725305.0
723504.0
725227.0
725831.0
724381.0
725725.0
723559.0
724763.0
724693.0
724978.0
724718.0
724732.0
724624.0
725198.0
725815.0
725385.0
724611.0
725308.0
723566.0
724891.0
723861.0
724401.0
723567.0
725224.0
724816.0
724375.0
725786.0
723826.0
725023.0
725244.0
725647.0
724262.0
725798.0
724459.0
725141.0
723840.0
724235.0
723992.0
725406.0
724530.0
723854.0
724148.0
724034.0
725271.0
725045.0
725529.0
725087.0
725897.0
724438.0
725475.0
717663.0
723533.0
724006.0
725478.0
725458.0
724853.0
725185.0
725826.0
724666.0
725310.0
724682.0
723795.0
725508.0
724672.0
725196.0
725058.0
724057.0
724638.0
723622.0
725046.0
724792.0
725477.0
724157.0
724783.0
723912.0
725330.0
725530.0
724974.0
725036.0
724243.0
724599.0
721477.0
725014.0
725288.0
724930.0
723749.0
723847.0
723661.0
724103.0
723638.0
725488.0
724258.0
724226.0
724242.0
724005.0
724876.0
723932.0
723709.0
7

In [114]:
subset_authors = data_subset_news_month.merge(authors, left_on='article_id', right_on='id')

subset_authors = subset_authors[['article_id', 'authors', 'Seitenaufrufe', 'published']]

subset_authors.head(50)

sub = subset_authors.groupby(['authors']).Seitenaufrufe.count()

sub.reset_index()

sub.to_csv('authors_article_sum.csv')

,article_id,authors,Seitenaufrufe,published
0,724751.0,Gabriel Brönnimann,30472.0,2016-07-15
1,725295.0,Jeremias Schulthess,18516.0,2016-07-22
2,724533.0,Renato Beck,13785.0,2016-07-13
3,725275.0,Matthias Oppliger,11986.0,2016-07-26
4,724826.0,Matthias Oppliger,10741.0,2016-07-16
5,725703.0,Gabriel Brönnimann,10112.0,2016-07-29
6,723646.0,Marc Krebs,8810.0,2016-07-01
7,723564.0,Yen Duong,7870.0,2016-07-01
8,725129.0,Renato Beck,7662.0,2016-07-20
9,724127.0,Jeremias Schulthess,6546.0,2016-07-08


,authors,Seitenaufrufe
0,Adrian Lobe,3
1,Adrian Schräder,1
2,Ali Farhat,1
3,Amir Mustedanagić,1
4,Andrea Fopp,3
5,Andreas Knobloch,1
6,Andreas Schneitter,2
7,Andreas Schwald,1
8,Antonia Brand,3
9,Astrid Frefel,2


In [87]:
subset_authors.tail()

,article_id,authors,Seitenaufrufe,published
303,728098.0,Christian Degen,63.0,2016-08-25
304,727495.0,Christian Degen,14.0,2016-08-18
305,727409.0,Renato Beck,10.0,2016-08-27
306,727409.0,Felix Michel,10.0,2016-08-27
307,727409.0,Stefan Kempf,10.0,2016-08-27


### Scrape für Article Types

In [ ]:
article_types = pd.read_csv('article_types.csv')

In [ ]:
article_types

In [ ]:
import urllib.request
import json
import codecs



def type_count(type):
    result = {}
    url = 'http://www.tageswoche.ch/content-api/articles?type=' + str(type)
    response = urllib.request.urlopen(url)
    reader = codecs.getreader("utf-8")
    obj = json.load(reader(response))
    result['type'] = type
    try:
        count = obj['pagination']['itemsCount']
        result['count'] = count
        print(count)
    except:
        result['count'] = 'weniger als 10'
        pass
    
    return pd.Series(result) #create a series

#type_count('static_page')

all_type_counts = article_types['types'].apply(type_count)

In [ ]:
all_type_counts

In [ ]:
all_type_counts.to_csv('type_count.csv')